In [1]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from sklearn.metrics import roc_curve, roc_auc_score, classification_report, accuracy_score
import numpy as np

import sys
import time
repo_root = "/data/quo.vadis/"
sys.path.append(repo_root)
from models import CompositeClassifier

/home/dmitrijs.trizna/.pyenv/versions/3.8-dev/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
def get_adversarial_samples(folder):
    fullpaths = [repo_root+folder+x for x in os.listdir(repo_root+folder)]
    adversarial_samples = [x for x in fullpaths if not os.path.islink(x)]
    adversarial_samples.sort()
    return adversarial_samples

In [17]:
ADVERSARIAL_EMULATED_SET_FOLDER = "data/adversarial.emulated/reports_ember_5sections_10population/"
ADVERSARIAL_RAW_SET_FOLDER = "data/adversarial.samples/samples_adversarial_testset_gamma_ember_sections/5/"

ADV_SAMPLES = get_adversarial_samples(ADVERSARIAL_RAW_SET_FOLDER)
ADV_SAMPLE_HASHES = [x.split("/")[-1] for x in ADV_SAMPLES]

classifier = CompositeClassifier(root=repo_root, emulation_report_path=ADVERSARIAL_EMULATED_SET_FOLDER,
                                rawpe_db_path=ADVERSARIAL_RAW_SET_FOLDER)

In [33]:
adversarial_emulated_files = os.listdir(repo_root + ADVERSARIAL_EMULATED_SET_FOLDER)
adversarial_reports = [x.replace(".json","") for x in adversarial_emulated_files if x.endswith(".json")]
ADV_REPORTS = [x for x in adversarial_reports if x in ADV_SAMPLE_HASHES]

### Verifying db objects

In [25]:
ex = ADV_SAMPLE_HASHES[1]
classifier.rawpe_db[ex]

'/data/quo.vadis/data/adversarial.samples/samples_adversarial_testset_gamma_ember_sections/5/00065eb186b596eea8d4dacfe971d9a00d2b35c20c06df84608f39d0d15d4dfc'

In [26]:
classifier.modules["filepaths"].filepath_db[ex]

'C:\\Windows\\Installer\\322d3e.exe (copy)'

In [39]:
for ex in ADV_REPORTS:
    classifier.modules["emulation"].report_db[ex]
classifier.modules["emulation"].report_db[ex]

'/data/quo.vadis/data/adversarial.emulated/reports_ember_5sections_10population/39300879444e5a4965e53b9fd5ba6b7fce73fc7f3585364c99011147cc9897a8.json'

In [22]:
# 2. ORIGINAL SAMPLES
classifier_orig = CompositeClassifier(root=repo_root)
classifier_orig.rawpe_db[ex]

'/data/quo.vadis/data/pe.dataset/PeX86Exe/dropper/00035f0b581f15721ace866f0920ceb5ff5ec16ee4cd4a9c3628316e05564804'

In [23]:
classifier_orig.modules["filepaths"].filepath_db[ex]

'C:\\PROGRA~3\\Mozilla\\ixxiuyl.exe'

In [28]:
classifier_orig.modules["emulation"].report_db[ex]

'/data/quo.vadis/data/emulation.dataset/report_dropper/00065eb186b596eea8d4dacfe971d9a00d2b35c20c06df84608f39d0d15d4dfc.json'

### All adversarial samples were successfully emulated in original case -- all of them are present in original report DB:

In [43]:
for ex in ADV_SAMPLE_HASHES:
    classifier_orig.modules["emulation"].report_db[ex]